In [1]:
import os
directory = r'D:\github\localt20\data\processedData'

import polars as pl
matches = pl.read_csv(os.path.join(directory, 'matches.csv'))
team_data = pl.read_csv(os.path.join(directory, 'teamStats.csv'))
matches

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/17""","""Melbourne Cricket Ground""",null,"""Sri Lanka""","""field""","""Sri Lanka""",1001349
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/19""","""Simonds Stadium, South Geelong""","""Victoria""","""Sri Lanka""","""field""","""Sri Lanka""",1001351
"""Australia""","""Sri Lanka""","""male""","""2016/17""","""2017/02/22""","""Adelaide Oval""",null,"""Sri Lanka""","""field""","""Australia""",1001353
"""Ireland""","""Hong Kong""","""male""","""2016""","""2016/09/05""","""Bready Cricket Club, Magherama…","""Londonderry""","""Hong Kong""","""bat""","""Hong Kong""",1004729
"""Zimbabwe""","""India""","""male""","""2016""","""2016/06/18""","""Harare Sports Club""",null,"""India""","""field""","""Zimbabwe""",1007655
…,…,…,…,…,…,…,…,…,…,…
"""Pakistan""","""Sri Lanka""","""male""","""2015/16""","""2016/03/04""","""Shere Bangla National Stadium""","""Mirpur""","""Pakistan""","""field""","""Pakistan""",966763
"""Bangladesh""","""India""","""male""","""2015/16""","""2016/03/06""","""Shere Bangla National Stadium""","""Mirpur""","""India""","""field""","""India""",966765
"""United Arab Emirates""","""Netherlands""","""male""","""2015/16""","""2016/02/03""","""ICC Academy""","""Dubai""","""United Arab Emirates""","""field""","""Netherlands""",967081


In [2]:
tdt = team_data['Team'].unique().to_list()
mdt = matches['team1'].unique().to_list()

In [3]:
for i in tdt:
    if i not in mdt:
        print(i)

Czech Rep.
U.S.A.
Peru
P.N.G.
ICC World XI
World-XI
Cayman
U.A.E.


In [4]:
for i in mdt:
    if i not in tdt:
        print(i)

United States of America
Papua New Guinea
Czech Republic
Cayman Islands
Barbados
United Arab Emirates


In [5]:
# Teams that need to be mapped between `tdt` and `mdt`
team_name_mapping = {
    'U.S.A.': 'United States of America',
    'U.A.E.': 'United Arab Emirates',
    'Czech Rep.': 'Czech Republic',
    'P.N.G.': 'Papua New Guinea',
    'Cayman': 'Cayman Islands'
}

# Teams that do not have a direct match
unmatched_tdt = [team for team in tdt if team not in mdt and team not in team_name_mapping]
unmatched_mdt = [team for team in mdt if team not in tdt and team not in team_name_mapping.values()]

print("Mapped Team Names Dictionary:", team_name_mapping)
print("Unmatched Teams in tdt:", unmatched_tdt)
print("Unmatched Teams in mdt:", unmatched_mdt)

Mapped Team Names Dictionary: {'U.S.A.': 'United States of America', 'U.A.E.': 'United Arab Emirates', 'Czech Rep.': 'Czech Republic', 'P.N.G.': 'Papua New Guinea', 'Cayman': 'Cayman Islands'}
Unmatched Teams in tdt: ['Peru', 'ICC World XI', 'World-XI']
Unmatched Teams in mdt: ['Barbados']


In [6]:
unmatched_teams = unmatched_tdt + unmatched_mdt
unmatched_teams

['Peru', 'ICC World XI', 'World-XI', 'Barbados']

In [7]:
print(team_data.shape, matches.shape)
team_data = team_data.filter(~pl.col('Team').is_in(unmatched_teams)).sort('Team')
matches = matches.filter(~pl.col('team1').is_in(unmatched_teams)).sort('team1')
matches = matches.filter(~pl.col('team2').is_in(unmatched_teams)).sort('team2')
print(team_data.shape, matches.shape)

(907, 13) (3683, 11)
(904, 13) (3676, 11)


In [8]:
team_name_mapping

{'U.S.A.': 'United States of America',
 'U.A.E.': 'United Arab Emirates',
 'Czech Rep.': 'Czech Republic',
 'P.N.G.': 'Papua New Guinea',
 'Cayman': 'Cayman Islands'}

In [9]:
team_data = team_data.with_columns(pl.col('Team').map_elements(lambda x: team_name_mapping.get(x, x),return_dtype=pl.String).alias('Team'))
matches = matches.with_columns(pl.col('team1').map_elements(lambda x: team_name_mapping.get(x, x),return_dtype=pl.String).alias('team1'))
matches = matches.with_columns(pl.col('team2').map_elements(lambda x: team_name_mapping.get(x, x),return_dtype=pl.String).alias('team2'))
matches

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id
str,str,str,str,str,str,str,str,str,str,i64
"""Australia""","""Afghanistan""","""male""","""2022/23""","""2022/11/04""","""Adelaide Oval""","""Adelaide""","""Afghanistan""","""field""","""Australia""",1298172
"""Bangladesh""","""Afghanistan""","""male""","""2018""","""2018/06/05""","""Rajiv Gandhi International Cri…","""Dehradun""","""Bangladesh""","""bat""","""Afghanistan""",1145983
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/15""","""Shere Bangla National Stadium,…","""Dhaka""","""Afghanistan""","""bat""","""Afghanistan""",1197142
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/21""","""Zahur Ahmed Chowdhury Stadium""","""Chattogram""","""Bangladesh""","""field""","""Bangladesh""",1197145
"""Bangladesh""","""Afghanistan""","""male""","""2021/22""","""2022/03/03""","""Shere Bangla National Stadium,…","""Dhaka""","""Bangladesh""","""bat""","""Bangladesh""",1299832
…,…,…,…,…,…,…,…,…,…,…
"""United Arab Emirates""","""Zimbabwe""","""male""","""2013/14""","""2014/03/21""","""Sylhet Stadium""",null,"""Zimbabwe""","""field""","""Zimbabwe""",682917
"""West Indies""","""Zimbabwe""","""male""","""2022/23""","""2022/10/19""","""Bellerive Oval, Hobart""","""Hobart""","""West Indies""","""bat""","""West Indies""",1298142
"""West Indies""","""Zimbabwe""","""male""","""2009/10""","""2010/02/28""","""Queen's Park Oval, Port of Spa…","""Trinidad""","""Zimbabwe""","""bat""","""Zimbabwe""",439139


In [10]:
data = matches.join(team_data, left_on=['team1','season'], right_on=['Team','Season'], how='inner',suffix='_team1')
data

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO
str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
"""Australia""","""Afghanistan""","""male""","""2022/23""","""2022/11/04""","""Adelaide Oval""","""Adelaide""","""Afghanistan""","""field""","""Australia""",1298172,9,5,3,0,1,1.666667,9,200,111.0,23.1,8.51
"""Bangladesh""","""Afghanistan""","""male""","""2018""","""2018/06/05""","""Rajiv Gandhi International Cri…","""Dehradun""","""Bangladesh""","""bat""","""Afghanistan""",1145983,6,2,4,0,0,0.5,6,184,122.0,20.88,7.54
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/15""","""Shere Bangla National Stadium,…","""Dhaka""","""Afghanistan""","""bat""","""Afghanistan""",1197142,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/21""","""Zahur Ahmed Chowdhury Stadium""","""Chattogram""","""Bangladesh""","""field""","""Bangladesh""",1197145,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85
"""Bangladesh""","""Afghanistan""","""male""","""2021/22""","""2022/03/03""","""Shere Bangla National Stadium,…","""Dhaka""","""Bangladesh""","""bat""","""Bangladesh""",1299832,13,3,10,0,0,0.3,13,181,73.0,16.88,6.66
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""United Arab Emirates""","""Zimbabwe""","""male""","""2013/14""","""2014/03/21""","""Sylhet Stadium""",null,"""Zimbabwe""","""field""","""Zimbabwe""",682917,3,0,3,0,0,0.0,3,151,151.0,15.6,6.51
"""West Indies""","""Zimbabwe""","""male""","""2022/23""","""2022/10/19""","""Bellerive Oval, Hobart""","""Hobart""","""West Indies""","""bat""","""West Indies""",1298142,8,3,5,0,0,0.6,8,258,118.0,22.35,8.85
"""West Indies""","""Zimbabwe""","""male""","""2009/10""","""2010/02/28""","""Queen's Park Oval, Port of Spa…","""Trinidad""","""Zimbabwe""","""bat""","""Zimbabwe""",439139,3,0,3,0,0,0.0,3,141,0.0,16.27,5.96


In [12]:
data = data.join(team_data, left_on=['team2','season'], right_on=['Team','Season'], how='inner',suffix='_team2')
data

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
"""Australia""","""Afghanistan""","""male""","""2022/23""","""2022/11/04""","""Adelaide Oval""","""Adelaide""","""Afghanistan""","""field""","""Australia""",1298172,9,5,3,0,1,1.666667,9,200,111.0,23.1,8.51,9,4,5,0,0,0.8,9,166,112.0,21.33,6.97
"""Bangladesh""","""Afghanistan""","""male""","""2018""","""2018/06/05""","""Rajiv Gandhi International Cri…","""Dehradun""","""Bangladesh""","""bat""","""Afghanistan""",1145983,6,2,4,0,0,0.5,6,184,122.0,20.88,7.54,5,5,0,0,0,5.0,5,167,0.0,23.24,7.92
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/15""","""Shere Bangla National Stadium,…","""Dhaka""","""Afghanistan""","""bat""","""Afghanistan""",1197142,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85,4,2,2,0,0,1.0,4,197,0.0,25.15,8.17
"""Bangladesh""","""Afghanistan""","""male""","""2019""","""2019/09/21""","""Zahur Ahmed Chowdhury Stadium""","""Chattogram""","""Bangladesh""","""field""","""Bangladesh""",1197145,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85,4,2,2,0,0,1.0,4,197,0.0,25.15,8.17
"""Bangladesh""","""Afghanistan""","""male""","""2021/22""","""2022/03/03""","""Shere Bangla National Stadium,…","""Dhaka""","""Bangladesh""","""bat""","""Bangladesh""",1299832,13,3,10,0,0,0.3,13,181,73.0,16.88,6.66,7,3,4,0,0,0.75,7,190,94.0,23.33,7.24
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""United Arab Emirates""","""Zimbabwe""","""male""","""2013/14""","""2014/03/21""","""Sylhet Stadium""",null,"""Zimbabwe""","""field""","""Zimbabwe""",682917,3,0,3,0,0,0.0,3,151,151.0,15.6,6.51,3,2,1,0,0,2.0,3,163,0.0,28.46,7.95
"""West Indies""","""Zimbabwe""","""male""","""2022/23""","""2022/10/19""","""Bellerive Oval, Hobart""","""Hobart""","""West Indies""","""bat""","""West Indies""",1298142,8,3,5,0,0,0.6,8,258,118.0,22.35,8.85,11,5,5,0,1,1.0,11,174,115.0,16.94,7.13
"""West Indies""","""Zimbabwe""","""male""","""2009/10""","""2010/02/28""","""Queen's Park Oval, Port of Spa…","""Trinidad""","""Zimbabwe""","""bat""","""Zimbabwe""",439139,3,0,3,0,0,0.0,3,141,0.0,16.27,5.96,1,1,0,0,0,1.0,1,105,105.0,10.5,5.29


In [13]:
data = data.with_columns(pl.when(pl.col("gender")=="male").then(1).otherwise(0).alias('gender'))
data.head()

team1,team2,gender,season,date,venue,city,toss_winner,toss_decision,winner,match_id,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
str,str,i32,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
"""Australia""","""Afghanistan""",1,"""2022/23""","""2022/11/04""","""Adelaide Oval""","""Adelaide""","""Afghanistan""","""field""","""Australia""",1298172,9,5,3,0,1,1.666667,9,200,111.0,23.1,8.51,9,4,5,0,0,0.8,9,166,112.0,21.33,6.97
"""Bangladesh""","""Afghanistan""",1,"""2018""","""2018/06/05""","""Rajiv Gandhi International Cri…","""Dehradun""","""Bangladesh""","""bat""","""Afghanistan""",1145983,6,2,4,0,0,0.5,6,184,122.0,20.88,7.54,5,5,0,0,0,5.0,5,167,0.0,23.24,7.92
"""Bangladesh""","""Afghanistan""",1,"""2019""","""2019/09/15""","""Shere Bangla National Stadium,…","""Dhaka""","""Afghanistan""","""bat""","""Afghanistan""",1197142,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85,4,2,2,0,0,1.0,4,197,0.0,25.15,8.17
"""Bangladesh""","""Afghanistan""",1,"""2019""","""2019/09/21""","""Zahur Ahmed Chowdhury Stadium""","""Chattogram""","""Bangladesh""","""field""","""Bangladesh""",1197145,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85,4,2,2,0,0,1.0,4,197,0.0,25.15,8.17
"""Bangladesh""","""Afghanistan""",1,"""2021/22""","""2022/03/03""","""Shere Bangla National Stadium,…","""Dhaka""","""Bangladesh""","""bat""","""Bangladesh""",1299832,13,3,10,0,0,0.3,13,181,73.0,16.88,6.66,7,3,4,0,0,0.75,7,190,94.0,23.33,7.24


In [14]:
# gender	match_id	Mat	Won	Lost	Tied	NR	W/L	Inns	HS	LS	AveRPW	AveRPO	Mat_team2	Won_team2	Lost_team2	Tied_team2	NR_team2	W/L_team2	Inns_team2	HS_team2	LS_team2	AveRPW_team2	AveRPO_team2
data = data.drop('team1','team2','season','date','venue','city','toss_winner','toss_decision','winner').select(['match_id','gender','Mat','Won','Lost','Tied','NR','W/L','Inns','HS','LS','AveRPW','AveRPO','Mat_team2','Won_team2','Lost_team2','Tied_team2','NR_team2','W/L_team2','Inns_team2','HS_team2','LS_team2','AveRPW_team2','AveRPO_team2'])
data

match_id,gender,Mat,Won,Lost,Tied,NR,W/L,Inns,HS,LS,AveRPW,AveRPO,Mat_team2,Won_team2,Lost_team2,Tied_team2,NR_team2,W/L_team2,Inns_team2,HS_team2,LS_team2,AveRPW_team2,AveRPO_team2
i64,i32,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,f64,i64,i64,f64,f64,f64
1298172,1,9,5,3,0,1,1.666667,9,200,111.0,23.1,8.51,9,4,5,0,0,0.8,9,166,112.0,21.33,6.97
1145983,1,6,2,4,0,0,0.5,6,184,122.0,20.88,7.54,5,5,0,0,0,5.0,5,167,0.0,23.24,7.92
1197142,1,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85,4,2,2,0,0,1.0,4,197,0.0,25.15,8.17
1197145,1,4,3,1,0,0,3.0,4,175,139.0,20.03,7.85,4,2,2,0,0,1.0,4,197,0.0,25.15,8.17
1299832,1,13,3,10,0,0,0.3,13,181,73.0,16.88,6.66,7,3,4,0,0,0.75,7,190,94.0,23.33,7.24
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
682917,1,3,0,3,0,0,0.0,3,151,151.0,15.6,6.51,3,2,1,0,0,2.0,3,163,0.0,28.46,7.95
1298142,1,8,3,5,0,0,0.6,8,258,118.0,22.35,8.85,11,5,5,0,1,1.0,11,174,115.0,16.94,7.13
439139,1,3,0,3,0,0,0.0,3,141,0.0,16.27,5.96,1,1,0,0,0,1.0,1,105,105.0,10.5,5.29


In [21]:
import numpy as np
directory = r'D:\github\localt20\finalData'

data.write_csv(os.path.join(directory, 'team12Stats.csv'))